# Project Pipeline

## 0. Import

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier

## 1. Tách dữ liệu để huấn luyện và kiểm thử
Mục đích của bước này là chia dữ liệu làm 2 phần, một phần để huấn luyện và phần còn lại coi như là thực tế và để đánh giá cuối cùng.
- Đầu vào: đường dẫn đến file dữ liệu đã crawl
- Đầu ra: 1 file train.csv và 1 file test.csv

In [2]:
def split_train_test(data_path, train_data_path, test_data_path):
    data_df = pd.read_csv(data_path, index_col=0, sep='\t')
    #data_df.dropna(inplace=True)
    data_df.label.astype(int)
    
    train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=0)
    train_df.to_csv(train_data_path)
    test_df.to_csv(test_data_path)

In [3]:
split_train_test('../data/csv/vietnamnews.csv', '../data/csv/train.csv', '../data/csv/test.csv')
data_df = pd.read_csv('../data/csv/train.csv', index_col = 0)

y_sr = data_df["label"]
X_df = data_df.drop("label", axis=1)

## 2. Thống kê dữ liệu tập huấn luyện
- Kiểm tra các giá trị thiếu
- Kiểm tra độ dài trung bình của nội dung và tiêu đề của các thể loại, và vẽ biểu đồ

In [4]:
def get_data_insight(X_df):
    raise NotImplementedError

ms = set()
for i, title in enumerate(X_df.title):
    prev_len = len(ms)
    ms.add(title)
    if prev_len == len(ms):
        print(title)
        

'ABBANK reports $36.97 million profit in first nine months'
'Park to coach Việt Nam until 2022'
'Experimental tuồng play to be staged in HCM\xa0City'
'New flight links Hải Phòng and Kunming'
'VNA making active contributions to OANA goal realisation'
'Liêm enters third round of FIDE World Cup'
'Hà Nội demolish Thanh Hoá in V.League'
'Solid waste treatment facilities to be examined'
'Foreign investors allowed 34 per cent holding at Vietnamese airlines'
'Việt Nam drawn against regional rivals for World Cup qualifiers'
'NA continues questioning ministers of industry and trade, home affairs'
Quick's Quick Tips
'Bắc Kạn manages mineral exploitation via cameras'
'Party Central Committee’s 10th plenum concludes'
'PM Phúc extends sympathy to China over natural disasters'
'Street foods test positive for\xa0carcinogens'


In [5]:
# TEST CELL

## 3. Tiền xử lý dữ liệu

In [42]:
# Module được thực hiện đầu Pipeline, xóa các khoảng trắng liên tiếp, các ký tự: \n, \t, ...

class RemoveIrrelevant(BaseEstimator, TransformerMixin):
    def clean_str(string):
        string = re.sub(r"\\xa0", "", string)
        string = re.sub(r"\'s", "", string)
        string = re.sub(r"\'ve", "", string)
        string = re.sub(r"n\'t", "", string)
        string = re.sub(r"\'re", "", string)
        string = re.sub(r"\'d", "", string)
        string = re.sub(r"\'ll", "", string)
        string = re.sub(r"'", "", string)
        string = re.sub(r"\\n", "", string)
        string = re.sub(r"\\t", "", string)
        return string.strip().lower()
    
    def fit(self, X_df, y=None):
        return self
    
    def transform(self, X_df, y=None):
        transform_X_df = X_df.copy()
        transform_X_df['title'] = transform_X_df['title'].apply(lambda x: clean_str(x))
        transform_X_df['text'] = transform_X_df['text'].apply(lambda x: clean_str(x))
        return transform_X_df

In [46]:
# TEST CELL
pl = RemoveIrrelevant()
transform_X = pl.transform(X_df)
print(X_df['text'][5])
print(transform_X['text'][5])

'MINSK —\xa0National Assembly (NA) Chairwoman Nguyễn Thị Kim Ngân and her entourage arrived at Minsk International Airport yesterday morning, beginning a three-day official visit to Belarus.The trip is made at the invitation of Chairman of the Council of the Republic Mikhail Myasnikovich and Chairman of the House of Representatives of the NA of\xa0Belarus Vladimir Andreichenko.\n\nWelcoming the Vietnamese delegation at the airport were Deputy Chairman of the Council of the Republic Isachenko Anatoly Mikhailovich, other officials of the Council, and Vietnamese Ambassador to Belarus Phạm Hải.\n\nThe first official trip to Belarus by Chairwoman Ngân aims to enhance the traditional friendship and multifaceted cooperation between the two countries and push ahead with the implementation of cooperation agreements between the two parliaments.\n\nIn 2019, the two countries’ relations have enjoyed strong developments with cooperation enhanced in all spheres.\n\nIn politics, Việt Nam and Belarus 

### 3.1.  Chọn các đặc trưng
+ Số ký tự văn bản
+ Số từ trong văn bản
+ Số lượng dấu cảm thán (!, ?)
+ Số chữ cái viết hoa
+ Số chữ cái viết thường
+ Các giá trị TF-IDF cho các từ trong văn bản (có xử lý văn bản trước đó), có những ưu tiên cho các từ trong title

#### 3.1.1 Các đặc trưng cơ bản

In [7]:
class NumCharFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass
    
class NumTokenFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass
    
    
class NumExclamationMarkFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass
    
class NumLowerCaseFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass

    
class NumUpperCaseFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass


#### 3.1.2 Thử nghiệm TF-IDF trên 2 cột title và text và đưa ra vài heuristic về sự ưu tiên cho cột title

In [8]:
# Các module tiền xử lý trước khi tính tf-idf

class RemoveSpace(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass
    
    
class RemoveTone(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass
    
class LowerCase(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass
        

In [9]:
# TF-IDF

## 4. Thiết kế mô hình hoàn chỉnh

### 4.1. Chia k fold 

In [10]:
def split_k_folds(n_train_sample, k):
    return cross_validation.KFold(n_train_sample, n_folds=k)

def split_k_folds2(k):
    return KFold(n_splits=k)

### 4.2. Khai báo các classifier

In [11]:
clf_list = []

# classifier parameters config
hyper_params_cfg = {
    'mlp':{
        'hidden_layer_sizes': [],
        'activation': [],
        'solver': []
    },
    'svm':{
        'param1': [],
        'param2': []
    }
}

### 4.3. Huấn luyện

In [12]:
# Khi train mới fit_transform và transform tại mỗi vòng for của fold

In [ ]:
def train_model(classifier, feature_vector_train, feature_vector_valid, label, is_mlp=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_mlp:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [ ]:
k_fold = split_k_folds2(k=5)
for train_index, valid_index in k_fold.split(X_df):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]
    
    